## 1. 合并数据框

合并数据框的常用函数有两个：pd.concat, pd.merge

pd.concat能同时操作多个数据框，pd.merge则提供类似SQL join的功能，专门用于合并两个数据框。

合并的前提条件是数据框具有相同的'key'，即'共同索引'，可以是行索引，也可以是列变量(默认)。

合并的方式有3种：

* 1对1合并：共同索引的值完全相同
* 多对1合并：其中一个数据框的共同索引有重复值
* 多对多合并：两个共同索引都有重复值

In [1]:
import pandas as pd

### 1.1 1对1合并

In [2]:
# 创建两个df，有相同的列变量'employee'，且列变量的值完全相同
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})

print(df1)
print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [3]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### 1.2 多对1合并

In [4]:
# 将df3和df4对比，有相同的列变量'group'，但df3的group有重复值
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})

print(df3)
print(df4)

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve


In [5]:
pd.merge(df3, df4)

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


### 1.3 多对多合并

In [6]:
# 将df1和df5对比，有相同的列变量'group'，且都有重复值
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})

print(df1)
print(df5)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization


In [7]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


## 2. 案例

根据2010年美国各州的人口密度进行排序。

下载数据，存储在同级目录的data文件夹。

In [9]:
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

In [55]:
pop = pd.read_csv("data/state-population.csv")
areas = pd.read_csv("data/state-areas.csv")
abbr = pd.read_csv("data/state-abbrevs.csv")

In [76]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [77]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [78]:
abbr.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


先合并'pop'和'abbr'两张表，然后合并'areas'，为了不漏掉任何数据，使用交集。

In [86]:
joined = pd.merge(pop, abbr, left_on="state/region", right_on="abbreviation", how="outer")
joined = pd.merge(joined, areas, on="state", how="outer")
joined.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
0,AL,under18,2012.0,1117489.0,Alabama,AL,52423.0
1,AL,total,2012.0,4817528.0,Alabama,AL,52423.0
2,AL,under18,2010.0,1130966.0,Alabama,AL,52423.0
3,AL,total,2010.0,4785570.0,Alabama,AL,52423.0
4,AL,under18,2011.0,1125763.0,Alabama,AL,52423.0


查看是否存在缺失值。

In [87]:
joined.loc[joined.isnull().any(axis=1)]

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
2448,PR,under18,1990.0,NaN,NaN,NaN,NaN
2449,PR,total,1990.0,NaN,NaN,NaN,NaN
2450,PR,total,1991.0,NaN,NaN,NaN,NaN
2451,PR,under18,1991.0,NaN,NaN,NaN,NaN
2452,PR,total,1993.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2540,USA,under18,2011.0,73902222.0,NaN,NaN,NaN
2541,USA,total,2011.0,311582564.0,NaN,NaN,NaN
2542,USA,under18,2012.0,73708179.0,NaN,NaN,NaN
2543,USA,total,2012.0,313873685.0,NaN,NaN,NaN


检查发现，PR州(即Puerto Rico)无人口数据，无法计算人口密度，而'USA'是国家名称，不仅没有面积数据，也不是我们关心的数据，故选择剔除所有缺失值。

In [88]:
joined.dropna(axis=0, inplace=True)
joined.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
0,AL,under18,2012.0,1117489.0,Alabama,AL,52423.0
1,AL,total,2012.0,4817528.0,Alabama,AL,52423.0
2,AL,under18,2010.0,1130966.0,Alabama,AL,52423.0
3,AL,total,2010.0,4785570.0,Alabama,AL,52423.0
4,AL,under18,2011.0,1125763.0,Alabama,AL,52423.0


再次检查是否还存在缺失值。

In [89]:
joined.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
abbreviation     False
area (sq. mi)    False
dtype: bool

计算人口密度，筛选2010年总人口密度，按从大到小的顺序排列。

In [93]:
joined["density"] = joined["population"] / joined["area (sq. mi)"]
joined = joined.query("year == 2010 & ages == 'total'")
joined.sort_values(by="density", ascending=False)

,state/region,ages,year,population,state,abbreviation,area (sq. mi),density
389,DC,total,2010.0,605125.0,District of Columbia,DC,68.0,8898.897059
1445,NJ,total,2010.0,8802707.0,New Jersey,NJ,8722.0,1009.253268
1914,RI,total,2010.0,1052669.0,Rhode Island,RI,1545.0,681.339159
293,CT,total,2010.0,3579210.0,Connecticut,CT,5544.0,645.600649
1050,MA,total,2010.0,6563263.0,Massachusetts,MA,10555.0,621.815538
965,MD,total,2010.0,5787193.0,Maryland,MD,12407.0,466.445797
379,DE,total,2010.0,899711.0,Delaware,DE,1954.0,460.445752
1541,NY,total,2010.0,19398228.0,New York,NY,54475.0,356.094135
475,FL,total,2010.0,18846054.0,Florida,FL,65758.0,286.597129
1829,PA,total,2010.0,12710472.0,Pennsylvania,PA,46058.0,275.966651
